**Google Analytics Data Wrangling**<br/>
Prof. Dr. Jan Kirenz <br/>
HdM Stuttgart, University of Applied Sciences

In [1]:
import re
import pandas as pd
from pandas.api.types import CategoricalDtype

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-data" data-toc-modified-id="Import-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import data</a></span></li><li><span><a href="#Tidying-data" data-toc-modified-id="Tidying-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Tidying data</a></span><ul class="toc-item"><li><span><a href="#Data-inspection" data-toc-modified-id="Data-inspection-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Data inspection</a></span></li><li><span><a href="#Data-cleaning-with-regular-expressions" data-toc-modified-id="Data-cleaning-with-regular-expressions-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data cleaning with regular expressions</a></span></li><li><span><a href="#Rename-variables" data-toc-modified-id="Rename-variables-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Rename variables</a></span></li><li><span><a href="#Change-data-types" data-toc-modified-id="Change-data-types-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Change data types</a></span></li><li><span><a href="#Check-data" data-toc-modified-id="Check-data-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Check data</a></span></li><li><span><a href="#Save-data" data-toc-modified-id="Save-data-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Save data</a></span></li></ul></li></ul></div>

Data cleaning of selected metrics from Google Analytics from the [Google merchandise account](https://support.google.com/analytics/answer/6367342#access). The data can be accessed [here](https://support.google.com/analytics/answer/6367342?hl=de).

## Import data

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/kirenz/datasets/master/google_merchandise_raw.csv", 
                 delimiter=',', skiprows=5, decimal=',')

## Tidying data

### Data inspection

First of all, let's take a look at the variables (columns) in the data set.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1220 entries, 0 to 1219
Data columns (total 15 columns):
Default Channel Grouping (Standard-Channelgruppierung)    1220 non-null object
Alter                                                     1220 non-null object
Land                                                      1220 non-null object
Absprungrate                                              1220 non-null object
Nutzer                                                    1220 non-null object
Seiten/Sitzung                                            1220 non-null float64
Durchschnittl. Sitzungsdauer                              1220 non-null object
Eindeutige Ereignisse                                     1220 non-null object
Hinzufügungen des Produkts zum Einkaufswagen              1220 non-null object
Purchase Completed (Abschlüsse für Zielvorhaben 1)        1220 non-null object
E-Commerce-Conversion-Rate                                1220 non-null object
Durchschn. Preis       

In [4]:
# show the first 5 rows (i.e. head of the DataFrame)
df.head(5)

,Default Channel Grouping (Standard-Channelgruppierung),Alter,Land,Absprungrate,Nutzer,Seiten/Sitzung,Durchschnittl. Sitzungsdauer,Eindeutige Ereignisse,Hinzufügungen des Produkts zum Einkaufswagen,Purchase Completed (Abschlüsse für Zielvorhaben 1),E-Commerce-Conversion-Rate,Durchschn. Preis,Durchschn. Bestellwert,Gesamtwert,Umsatz
0,Display,65+,Mexico,"100,00 %",79,1.00,00:00:00,0,0,0,"0,00 %","0,00 $","0,00 $","0,00 $","0,00 $"
1,Display,65+,Canada,"96,65 %",188,1.17,00:00:04,22,0,0,"0,00 %","0,00 $","0,00 $","0,00 $","0,00 $"
2,Social,55-64,Italy,"94,23 %",231,1.11,00:00:08,22,0,0,"0,00 %","0,00 $","0,00 $","0,00 $","0,00 $"
3,Display,35-44,Saudi Arabia,"93,50 %",101,1.11,00:00:04,0,0,0,"0,00 %","0,00 $","0,00 $","0,00 $","0,00 $"
4,Display,25-34,Argentina,"92,02 %",79,1.07,00:00:48,0,0,0,"0,00 %","0,00 $","0,00 $","0,00 $","0,00 $"


In [5]:
# show all variables in the data set
df.columns

Index(['Default Channel Grouping (Standard-Channelgruppierung)', 'Alter',
       'Land', 'Absprungrate', 'Nutzer', 'Seiten/Sitzung',
       'Durchschnittl. Sitzungsdauer', 'Eindeutige Ereignisse',
       'Hinzufügungen des Produkts zum Einkaufswagen',
       'Purchase Completed (Abschlüsse für Zielvorhaben 1)',
       'E-Commerce-Conversion-Rate', 'Durchschn. Preis',
       'Durchschn. Bestellwert', 'Gesamtwert', 'Umsatz'],
      dtype='object')

### Data cleaning with regular expressions

In [6]:
# Select relevant columns for data cleaning
cols = ['Absprungrate', 
        'E-Commerce-Conversion-Rate', 
        'Durchschn. Preis', 
        'Durchschn. Bestellwert', 
        'Gesamtwert', 'Umsatz']

# Remove thousand sep (.), replace decimal symbol, remove unit symbol and whitespace
df = df.replace({'\.':'', ',':'.', '\s*%\s*':'','\s*\$\s*':''}, regex=True)  

# Convert Datatype
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

### Rename variables

In [7]:
# Rename Variables
df = df.rename(index=str, 
               columns={"Default Channel Grouping (Standard-Channelgruppierung)": "Channel", 
                        "Durchschnittl. Sitzungsdauer": "Sitzungsdauer",
                        "Hinzufügungen des Produkts zum Einkaufswagen": "Einkaufswagen",
                        "Purchase Completed (Abschlüsse für Zielvorhaben 1)": "Purchase_Completed",
                        "E-Commerce-Conversion-Rate": "Conversion_Rate",
                        "Durchschn. Preis": "Preis",
                        "Durchschn. Bestellwert": "Betsellwert"})

### Change data types

In [8]:
df['Alter'].value_counts() 

25-34    341
18-24    268
35-44    266
45-54    157
55-64    104
65+       84
Name: Alter, dtype: int64

In [9]:
# convert to ordinal variable
cat_Alter = CategoricalDtype(categories=
                            ['18-24', 
                             '25-34', 
                             '35-44', 
                             '45-54', 
                             '55-64',
                             '65+'],
                            ordered=True)

df.Alter = df.Alter.astype(cat_Alter)

In [10]:
# change data types 
df.iloc[:,0] = pd.Categorical(df.iloc[:,0])
df.iloc[:,1] = pd.Categorical(df.iloc[:,1])
df.iloc[:,2] = pd.Categorical(df.iloc[:,2])
df.iloc[:,3] = pd.to_numeric(df.iloc[:,3])
df.iloc[:,4] = pd.to_numeric(df.iloc[:,4])
df.iloc[:,5] = pd.to_numeric(df.iloc[:,5])
df.iloc[:,7] = pd.to_numeric(df.iloc[:,7])
df.iloc[:,8] = pd.to_numeric(df.iloc[:,8])
df.iloc[:,9] = pd.to_numeric(df.iloc[:,9])
df.iloc[:,10] = pd.to_numeric(df.iloc[:,10])
df.iloc[:,11] = pd.to_numeric(df.iloc[:,11])
df.iloc[:,12] = pd.to_numeric(df.iloc[:,12])
df.iloc[:,13] = pd.to_numeric(df.iloc[:,13])
df.iloc[:,14] = pd.to_numeric(df.iloc[:,14])

### Check data

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1220 entries, 0 to 1219
Data columns (total 15 columns):
Channel                  1220 non-null category
Alter                    1220 non-null category
Land                     1220 non-null category
Absprungrate             0 non-null float64
Nutzer                   1220 non-null float64
Seiten/Sitzung           1220 non-null float64
Sitzungsdauer            1220 non-null object
Eindeutige Ereignisse    1220 non-null float64
Einkaufswagen            1220 non-null float64
Purchase_Completed       1220 non-null float64
Conversion_Rate          0 non-null float64
Preis                    0 non-null float64
Betsellwert              0 non-null float64
Gesamtwert               0 non-null float64
Umsatz                   0 non-null float64
dtypes: category(3), float64(11), object(1)
memory usage: 133.9+ KB


### Save data

In [12]:
# Save clean data
# df.to_csv('google_merchandise.csv')